# 0. Standard imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('WebAgg')
import numpy as np
import pandas as pd

In [2]:
cd ..

/home/eecs/khalil.ouardini/Cassiopeia_Transcriptome/scvi/external


In [3]:
cd ..

/home/eecs/khalil.ouardini/Cassiopeia_Transcriptome/scvi


### import ete3 Tree

In [4]:
from ete3 import Tree

tree_name = "/data/yosef2/users/mattjones/projects/metastasis/JQ19/5k/trees/lg7/lg7_tree_hybrid_priors.alleleThresh.processed.txt"
tree = Tree(tree_name, 1)

N = len([n for n in tree.traverse()])

leaves = [n.name for n in tree.traverse('levelorder') if n.is_leaf()]

print('{} nodes in the tree'.format(N))
print('{} leaves in the tree'.format(len(leaves)))

2247 nodes in the tree
663 leaves in the tree


In [5]:
print(tree)

    |  |     |  |
           |  |     |  |   /- /- /- /- /-M1.ACTGATGAGGTCGGAT-1
           |  |     |  |  |
           |  |     |  |--|   /- /- /-M1.GATCAGTTCTGTTGAG-1
           |  |     |  |  |  |
           |  |     |  |   \-|-- /-RE.TCACAAGTCCCAACGG-1
           |  |     |  |     |
           |  |     |  |      \- /-M2.CCCAATCCAGGTTTCA-1
           |  |     |  |
           |  |     |  |   /- /- /- /-M1.GACGTGCTCACCGGGT-1
           |  |     |  |--|
           |  |     |  |  |   /- /-M1.TTGCCGTGTTCGTGAT-1
           |  |     |  |   \-|
           |  |     |  |      \- /- /-RE.TCAATCTAGGGTTTCT-1
           |  |     |  |
           |  |     |  |-- /- /- /- /-LL.GATGAGGAGAGCAATT-1
           |  |     |  |
           |  |     |  |-- /- /- /-M2.GCATGCGGTCTCATCC-1
           |  |     |  |
           |  |     |  |   /- /- /- /-M2.TACTTGTTCCACTCCA-1
           |  |     |  |--|
           |  |     |  |  |         /- /- /-M1.CATTCGCAGCCTTGAT-1
           |  |     |  |   \- /- /-|
           

### Gene expression data

In [10]:
barcodes = pd.read_csv("/data/yosef2/users/mattjones/projects/metastasis/JQ19/5k/RNA/ALL_Samples/GRCh38/barcodes.tsv", names=['Barcodes'])

barcodes.head(5)

,Barcodes
0,LL.AAACCTGCAAGAAAGG-1
1,LL.AAACCTGCATGAAGTA-1
2,LL.AAACCTGGTACATGTC-1
3,LL.AAACCTGTCAAGGTAA-1
4,LL.AAACGGGGTCTTGATG-1


In [11]:
genes = pd.read_csv("/data/yosef2/users/mattjones/projects/metastasis/JQ19/5k/RNA/ALL_Samples/GRCh38/genes.tsv", sep='\t', names=['id', 'Gene'])

genes.head(5)

,id,Gene
0,ENSG00000243485,RP11-34P13.3
1,ENSG00000237613,FAM138A
2,ENSG00000186092,OR4F5
3,ENSG00000238009,RP11-34P13.7
4,ENSG00000239945,RP11-34P13.8


In [12]:
hotspot_genes = pd.read_csv("/data/yosef2/users/mattjones/projects/metastasis/hotspot_gene_sets/lg7_hotspot_genes.tsv", sep='\t')
hotspot_genes.head(5)

,Gene,C,Z,Pval,FDR
0,TFF3,0.174633,42.425806,0.000000e+00,0.000000e+00
1,TFF1,0.227374,41.599879,0.000000e+00,0.000000e+00
2,MUC5AC,0.157811,37.940520,0.000000e+00,0.000000e+00
3,CEACAM6,0.245428,32.255498,1.472918e-228,2.218583e-225
4,MUC5B,0.084600,24.601674,6.060690e-134,7.303131e-131


Gene expression

In [18]:
from scipy.io import mmread

X = mmread('/data/yosef2/users/mattjones/projects/metastasis/JQ19/5k/RNA/ALL_Samples/GRCh38/matrix.mtx')

<33694x43423 sparse matrix of type '<class 'numpy.int64'>'
	with 90063434 stored elements in COOrdinate format>

In [19]:
Y = X.todense()

In [87]:
L = barcodes['Barcodes'].values

In [88]:
len(L)

43423

In [81]:
"M2.TAGCCGGAGGGCTTCC-1" in L

False

In [89]:
leaves_X = []
for barcode in leaves:
    foo = barcodes.index[barcodes['Barcodes'] == barcode].tolist()
    if foo == []:
        print(barcode)
    #idx = barcodes.index[barcodes['Barcodes'] == barcode].tolist()[0]
    x = np.squeeze(np.array(Y[:, idx]))
    leaves_X.append(x)
#leaves_X.shape

RE.TAAGCGTTCTAACTCT-1
RE.CGGACACAGGAGCGTT-1
RE.TGGTTCCCACCGAATT-1
RE.GATCGATAGGGATGGG-1
RE.CGTTGGGAGCACGCCT-1
RE.GCATGCGCAGGAATCG-1
RE.CTGTTTACACCCATGG-1
RE.GCACATAGTACGCTGC-1
RE.CGTAGGCCACGAGAGT-1
RE.GTGCAGCAGGTTCCTA-1
RE.CGTGTAATCGAATGCT-1
RE.CCTCTGATCTGCGACG-1
RE.CGTAGGCCAACACCCG-1
RE.CGTGTAAAGATAGCAT-1
RE.GTAGGCCAGCGTCAAG-1
RE.CAGTAACTCCTGTAGA-1
RE.CGATGTACACAGAGGT-1
RE.ATCTACTCAGTCGTGC-1
RE.GCTTCCATCCCACTTG-1
RE.ATGGGAGAGTCATGCT-1
RE.TTCCCAGAGGGTTTCT-1
RE.ACTGAGTGTCATGCAT-1
RE.AACACGTTCTCGATGA-1
RE.CTACACCTCATAAAGG-1
RE.CGGACGTAGCGAGAAA-1
RE.CTTACCGTCCAAACTG-1
RE.TGACTTTAGCACGCCT-1
M1.TAGAGCTAGGCTAGCA-1
RE.TGAGCCGGTACGAAAT-1
RE.CGTAGCGGTCAGTGGA-1
RE.GACTGCGGTTACGACT-1
RE.GATCGATAGCCCAGCT-1
RE.TCAGCTCCACATGTGT-1
RE.CTTCTCTGTAGCACGA-1
RE.TGACTAGTCCGAATGT-1
RE.CTTTGCGAGCGTAATA-1
RE.TGAAAGAAGTCTCGGC-1
RE.ACTTGTTTCGTCACGG-1
RE.GCCTCTATCTTGACGA-1
RE.CACAGTAGTATCACCA-1
RE.CGCCAAGAGTCCGGTC-1
RE.ACGGGCTCAAGTTCTG-1
M2.GAACCTATCCAATGGT-1
RE.CAGCCGAGTTCGAATC-1
RE.CAGCTAAGTCAGAATA-1
RE.GTCAAGT

In [ ]:
for i, n in enumerate(tree.traverse('levelorder')):
    n.add_features(index=i)
    if not n.is_leaf():
        n.name = str(i)

In [ ]:
branch_length = {}
for node in tree.traverse('levelorder'):
    if node.name == '0':
        branch_length[node.name] = 0.0
        continue
    branch_length[node.name] = 1.0
branch_length['prior_root'] = 1.0

In [209]:
#for i, n in enumerate(tree.traverse()):
#    print(i, n.name)

#split_node = tree.search_nodes(name='2|2|2|2|2|0|2|2|2|0|2|2|0|2|0|2|2|2|0|2|2|2|2|0|0|0|0|0|0')[0]
#sub_leaves = [n.name for n in split_node.get_leaves()]
#tree.prune(sub_leaves)

In [210]:
# Data
from anndata import AnnData
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from external.dataset.tree import TreeDataset, GeneExpressionDataset
from external.dataset.poisson_glm import Poisson_GLM
from external.dataset.anndataset import AnnDatasetFromAnnData

# Models
from models.vae import VAE
import scanpy as sc
from external.inference.tree_inference import TreeTrainer
from inference.inference import UnsupervisedTrainer
from scvi.inference import posterior
from external.models.treevae import TreeVAE

# Utils
from external.utils.data_util import get_leaves, get_internal
from external.utils.metrics import ks_pvalue, accuracy_imputation, correlations, knn_purity, knn_purity_stratified
from external.utils.plots_util import plot_histograms, plot_scatter_mean, plot_ecdf_ks, plot_density
from external.utils.plots_util import plot_losses, plot_elbo, plot_common_ancestor, plot_one_gene, training_dashboard
from external.utils.baselines import avg_weighted_baseline, scvi_baseline, scvi_baseline_z, cascvi_baseline_z, avg_baseline_z, construct_latent

In [211]:
import torch
    
np.random.seed(42)
torch.manual_seed(42)

***Simulated latent space***

In [232]:
#plot_common_ancestor(tree,
#                     glm.z,
#                     embedding='umap',
#                     give_labels=False
#                     )

# 2. Fitting CascVI

In [233]:
import scanpy as sc

# anndata + gene and celle filtering
adata = AnnData(leaves_X)
leaves = [n for n in tree.traverse('levelorder') if n.is_leaf()]
adata.obs_names = [n.name for n in leaves]
sc.pp.filter_genes(adata, min_counts=3)
sc.pp.filter_cells(adata, min_counts=0)

***Create a TreeDataset object***

In [234]:
# treeVAE
import copy

tree_bis = copy.deepcopy(tree)
scvi_dataset = AnnDatasetFromAnnData(adata, filtering=False)
scvi_dataset.initialize_cell_attribute('barcodes', adata.obs_names)
cas_dataset = TreeDataset(scvi_dataset, tree=tree_bis, filtering=False)
cas_dataset

# No batches beacause of the message passing
use_cuda = True
use_MP = True
ldvae = False

go
[2021-05-10 13:55:18,075] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2021-05-10 13:55:18,077] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2021-05-10 13:55:18,103] INFO - scvi.dataset.dataset | Merging datasets. Input objects are modified in place.
[2021-05-10 13:55:18,104] INFO - scvi.dataset.dataset | Gene names and cell measurement names are assumed to have a non-null intersection between datasets.
[2021-05-10 13:55:18,106] INFO - scvi.dataset.dataset | Keeping 1000 genes
[2021-05-10 13:55:18,116] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2021-05-10 13:55:18,118] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2021-05-10 13:55:18,119] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2021-05-10 13:55:18,122] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2021-05-10 13:55:18,123] INFO - scvi.dataset.dataset | Remapping labels to [0,N]


***Initialize model***

In [235]:
treevae = TreeVAE(cas_dataset.nb_genes,
              tree = cas_dataset.tree,
              n_latent=glm.latent,
              n_hidden=128,
              n_layers=1,
              reconstruction_loss='poisson',
              prior_t = branch_length,
              ldvae = ldvae,
              use_MP=use_MP
             )

In [236]:
import torch

freeze = False
if freeze:
    new_weight = torch.from_numpy(glm.W).float()
    new_bias = torch.from_numpy(glm.beta).float()

    with torch.no_grad():
        treevae.decoder.factor_regressor.fc_layers[0][0].weight = torch.nn.Parameter(new_weight)
        treevae.decoder.factor_regressor.fc_layers[0][0].bias = torch.nn.Parameter(new_bias)
        
    for param in treevae.decoder.factor_regressor.fc_layers[0][0].parameters():
        param.requires_grad = False
    
treevae.decoder

Decoder(
  (px_decoder): FCLayers(
    (fc_layers): Sequential(
      (Layer 0): Sequential(
        (0): Linear(in_features=10, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): None
      )
    )
  )
  (px_scale_decoder): Sequential(
    (0): Linear(in_features=128, out_features=1000, bias=True)
    (1): Softmax(dim=-1)
  )
)

In [237]:
#assert(treevae.decoder.factor_regressor.fc_layers[0][0].weight.numpy().all() == glm.W.T.all())
#assert(treevae.decoder.factor_regressor.fc_layers[0][0].bias.numpy().all() == glm.beta.all())

***Are we able to generate the gene expression data by decoding the simulated latent space?***

In [238]:
px_scale, px_rate, raw_px_scale = treevae.decoder(treevae.dispersion,
                                        torch.from_numpy(leaves_z).float(),
                                        torch.from_numpy(np.array([np.log(10000)])).float()
                                       )

from sklearn.metrics import mean_squared_error

if ldvae:
    foo = np.clip(a=np.exp(raw_px_scale.detach().cpu().numpy()),
            a_min=0,
            a_max=1e8
    )
    mse = mean_squared_error(mu, foo)
else:
    mse = mean_squared_error(mu, px_rate.detach().numpy())

print("the distance between the Poisson and the NB means is {}".format(mse))

the distance between the Poisson and the NB means is 44813.85865894124


***Hyperparameters***

In [239]:
n_epochs = 1000
lr = 1e-3
lambda_ = 10.0

***trainer***

In [240]:
freq = 100
trainer = TreeTrainer(
    model = treevae,
    gene_dataset = cas_dataset,
    lambda_ = lambda_,
    train_size=1.0,
    test_size=0,
    use_cuda=use_cuda,
    frequency=freq,
    n_epochs_kl_warmup=300
)

train_leaves:  [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68], [69], [70], [71], [72], [73], [74], [75], [76], [77], [78], [79], [80], [81], [82], [83], [84], [85], [86], [87], [88], [89], [90], [91], [92], [93], [94], [95], [96], [97], [98], [99], [100], [101], [102], [103], [104], [105], [106], [107], [108], [109], [110], [111], [112], [113], [114], [115], [116], [117], [118], [119], [120], [121], [122], [123], [124], [125], [126], [127], [128], [129], [130], [131], [132], [133], [134], [135], [136], [137], [138], [139], [140], [141], [142], [143], [144], [145], [146], [147], [148], [149], [150], [151], [152], [153], [154], [155], [1

***Start training***

In [241]:
trainer.train(n_epochs=n_epochs,
              lr=lr
              )

0688256816
training:  85%|████████▌ | 850/1000 [03:07<00:35,  4.24it/s]Encodings MP Likelihood: 1.0250695639118177
ELBO Loss: 561.5198904665988
training:  85%|████████▌ | 851/1000 [03:07<00:39,  3.78it/s]Encodings MP Likelihood: 0.8278356571127333
ELBO Loss: 566.2978840524001
training:  85%|████████▌ | 852/1000 [03:07<00:38,  3.86it/s]Encodings MP Likelihood: 1.0835691405472634
ELBO Loss: 563.7063535346398
training:  85%|████████▌ | 853/1000 [03:08<00:37,  3.94it/s]Encodings MP Likelihood: 1.1772286054600822
ELBO Loss: 567.0721246901841
training:  85%|████████▌ | 854/1000 [03:08<00:37,  3.93it/s]Encodings MP Likelihood: 1.9331837257358977
ELBO Loss: 569.4242327883079
training:  86%|████████▌ | 855/1000 [03:08<00:36,  4.00it/s]Encodings MP Likelihood: 1.0168637288368112
ELBO Loss: 564.6312752131631
training:  86%|████████▌ | 856/1000 [03:08<00:35,  4.07it/s]Encodings MP Likelihood: 1.6080092162429296
ELBO Loss: 565.4173610547424
training:  86%|████████▌ | 857/1000 [03:09<00:34,  4.11it/

***Loss Functions***

In [242]:
training_dashboard(trainer, treevae.encoder_variance)

### 3. Posterior and MV imputation

In [243]:
from sklearn.metrics import mean_squared_error

full_posterior = trainer.create_posterior(trainer.model, cas_dataset, trainer.clades,
                                indices=np.arange(len(cas_dataset))
                                         )
error = mean_squared_error(full_posterior.get_latent(), leaves_z)
print("the distance is {}".format(error))

the distance is 1.5378348446839363


***Missing Value imputation By Posterior Predictive sampling***

In [171]:
empirical_l = np.mean(np.sum(glm.X, axis=1))

# CascVI impitations
imputed = {}
imputed_z = {}
imputed_gt = {}

for n in tree.traverse('levelorder'):
    if not n.is_leaf():
        imputed[n.name], imputed_z[n.name] = full_posterior.imputation_internal(n.name,
                                                            give_mean=False,
                                                            library_size=empirical_l
                                                           )
        imputed_gt[n.name] = glm.X[n.index]

In [172]:
imputed_X = [x for x in imputed.values()]
imputed_X = np.array(imputed_X).reshape(-1, cas_dataset.X.shape[1])
#plot_histograms(imputed_X, "Histogram of CasscVI imputed gene expression data")

***CascVI Baseline 1 (MP Oracle)***

In [173]:
# CascVI impitations
imputed_cascvi_1 = {}
imputed_cascvi_1_z ={}

for n in tree.traverse('levelorder'):
    if not n.is_leaf():
        _, imputed_cascvi_1_z[n.name] = full_posterior.imputation_internal(n.name,
                                                                    give_mean=False,
                                                                    library_size=empirical_l,
                                                                    known_latent=leaves_z
        )
        mu_z = np.clip(a=np.exp(glm.W @ imputed_cascvi_1_z[n.name].cpu().numpy() + glm.beta),
                        a_min=0,
                        a_max=1e8
                        )
        samples = np.array([np.random.poisson(mu_z) for i in range(100)])
        imputed_cascvi_1[n.name] = np.clip(a=np.mean(samples, axis=0),
                                           a_min=0,
                                           a_max=1e8
                                           )


***CascVI Baseline 2 (Reconstruction of Averaged latent space)***

In [174]:
imputed_cascvi_2, imputed_cascvi_2_z, _ = avg_baseline_z(tree=tree,
                                   model=treevae,
                                   posterior=full_posterior,
                                   weighted=False,
                                   n_samples_z=1,
                                   library_size=empirical_l,
                                   gaussian=False,
                                   use_cuda=True
                                  )

# 4. Baselines

### Baseline 1: Unweighted Average of gene expression in Clade

The simple idea here is to impute the value of an internal node, with the (un)weighted average of the gene expression values of the leaves, taking the query internal node as the root of the subtree.

In [175]:
weighted = False
imputed_avg = avg_weighted_baseline(tree, weighted, glm.X, rounding=True)

#get internal nodes
avg_X = np.array([x for x in imputed_avg.values()]).reshape(-1, glm.X.shape[1])
internal_avg_X, _, _ = get_internal(avg_X, glm.mu, tree)

### Baseline 2: (Un)weighted Average of decoded latent vectors, with scVI

We use the same averaging of the subtrees leaves in **Baseline 1**, only this time, the gene expression data is recovered with scVI

In [176]:
# anndata
gene_dataset = GeneExpressionDataset()
gene_dataset.populate_from_data(leaves_X)

[2021-05-10 13:50:15,165] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2021-05-10 13:50:15,167] INFO - scvi.dataset.dataset | Remapping labels to [0,N]


In [177]:
import torch

n_epochs =800
use_batches = False

vae = VAE(gene_dataset.nb_genes,
                  n_batch=cas_dataset.n_batches * use_batches,
                  n_hidden=128,
                  n_layers=1,
                  reconstruction_loss='poisson',
                  n_latent=glm.latent,
                  ldvae=ldvae
              )

if freeze:
    new_weight = torch.from_numpy(glm.W).float()
    new_bias = torch.from_numpy(glm.beta).float()

    with torch.no_grad():
        vae.decoder.factor_regressor.fc_layers[0][0].weight = torch.nn.Parameter(new_weight)
        vae.decoder.factor_regressor.fc_layers[0][0].bias = torch.nn.Parameter(new_bias)
        
    for param in vae.decoder.factor_regressor.fc_layers[0][0].parameters():
        param.requires_grad = False
    
vae.decoder

DecoderSCVI(
  (px_decoder): FCLayers(
    (fc_layers): Sequential(
      (Layer 0): Sequential(
        (0): Linear(in_features=10, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): None
      )
    )
  )
  (px_scale_decoder): Sequential(
    (0): Linear(in_features=128, out_features=1000, bias=True)
    (1): Softmax(dim=-1)
  )
  (px_r_decoder): Linear(in_features=128, out_features=1000, bias=True)
  (px_dropout_decoder): Linear(in_features=128, out_features=1000, bias=True)
)

In [178]:
px_scale, px_r, px_rate, px_dropout = vae.decoder.forward(vae.dispersion,
                                        torch.from_numpy(leaves_z).float(),
                                        torch.from_numpy(np.array([np.log(10000)])).float(),
                                        None
                                        )

from sklearn.metrics import mean_squared_error



if ldvae:
    foo = np.clip(a=np.exp(px_r.detach().numpy()),
            a_min=0,
            a_max=5000
    )
    mse = mean_squared_error(mu, foo)
else:
    mse = mean_squared_error(mu, px_rate.detach().numpy())

print("the distance between the Poisson and the NB means is {}".format(mse))

the distance between the Poisson and the NB means is 1463076.6425772563


In [179]:
trainer_scvi = UnsupervisedTrainer(model=vae,
                              gene_dataset=gene_dataset,
                              train_size=1.0,
                              use_cuda=use_cuda,
                              frequency=10,
                              n_epochs_kl_warmup=200)

# train scVI
trainer_scvi.train(n_epochs=n_epochs, lr=1e-3) 
                                        
elbo_train_scvi = trainer_scvi.history["elbo_train_set"]
x = np.linspace(0, 100, (len(elbo_train_scvi)))
plt.plot(np.log(elbo_train_scvi), 
         label="train", color='blue',
         linestyle=':',
         linewidth=3
        )
        
plt.xlabel('Epoch')
plt.ylabel("ELBO")
plt.legend()
plt.title("Train history scVI")
plt.show()

training: 100%|██████████| 800/800 [00:48<00:00, 16.54it/s]


In [180]:
scvi_posterior = trainer_scvi.create_posterior(model=vae,
                                               gene_dataset=gene_dataset 
                                                )

error = mean_squared_error(scvi_posterior.get_latent()[0], leaves_z)
print("the distance is {}".format(error))

the distance is 8.892578060195763


***scVI Baseline 2 (Decoded Average Latent space)***

In [181]:
library_size = np.mean(np.sum(glm.X, axis=1))
scvi_latent = np.array([scvi_posterior.get_latent(give_mean=False)[0] for i in range(10)])

imputed_scvi_2, imputed_scvi_2_z = scvi_baseline_z(tree,
                                        posterior=scvi_posterior,
                                        model=vae,
                                        weighted=False,
                                        n_samples_z=1,
                                        library_size=library_size,
                                        use_cuda=True)


# 5. Likelihood Ratio

In [182]:
cascvi_latent = full_posterior.get_latent()
scvi_latent = scvi_posterior.get_latent()[0]

scvi_latent.shape, cascvi_latent.shape

((500, 10), (500, 10))

In [183]:
treevae.initialize_visit()
treevae.initialize_messages(scvi_latent, cas_dataset.barcodes, scvi_latent.shape[1])
treevae.perform_message_passing((treevae.tree & treevae.root), scvi_latent.shape[1], False)
mp_lik_scvi = treevae.aggregate_messages_into_leaves_likelihood(d, add_prior=True)
print("Likelihood of scVI encodings: ", mp_lik_scvi.item())

Likelihood of scVI encodings:  -89983.96007352648


In [184]:
treevae.initialize_visit()
treevae.initialize_messages(cascvi_latent, cas_dataset.barcodes, cascvi_latent.shape[1])
treevae.perform_message_passing((treevae.tree & treevae.root), cascvi_latent.shape[1], False)
mp_lik_cascvi = treevae.aggregate_messages_into_leaves_likelihood(d, add_prior=True)
print("Likelihood of cascVI encodings: ", mp_lik_cascvi.item())

Likelihood of cascVI encodings:  -3151.64372982798


In [185]:
treevae.initialize_visit()
treevae.initialize_messages(leaves_z, cas_dataset.barcodes, cascvi_latent.shape[1])
treevae.perform_message_passing((treevae.tree & treevae.root), cascvi_latent.shape[1], False)
mp_lik_cascvi = treevae.aggregate_messages_into_leaves_likelihood(d, add_prior=True)
print("Likelihood of observations: ", mp_lik_cascvi.item())

Likelihood of observations:  -11128.761668907146


In [186]:
# Likelihood ratio
lambda_ = (mp_lik_cascvi - mp_lik_scvi)
print("Likelihood Ratio:", lambda_)

Likelihood Ratio: tensor(78855.1984, dtype=torch.float64)


# 6. Evaluation

***CPM Normalization (for sample-sample correlation)***

get imputations into an array

In [187]:
internal_scvi_X_2 = np.array([x for x in imputed_scvi_2.values()]).reshape(-1, glm.X.shape[1])
internal_cascvi_X = np.array([x for x in imputed_cascvi_1.values()]).reshape(-1, glm.X.shape[1])
internal_cascvi_X_2 = np.array([x for x in imputed_cascvi_2.values()]).reshape(-1, glm.X.shape[1])

internal_cascvi_X_2.shape, internal_cascvi_X.shape, internal_scvi_X_2.shape, imputed_X.shape, internal_avg_X.shape, internal_X.shape

((500, 1000), (500, 1000), (500, 1000), (500, 1000), (500, 1000), (500, 1000))

In [188]:
from sklearn.preprocessing import normalize

norm_internal_X = sc.pp.normalize_total(AnnData(internal_X), target_sum=1e4, inplace=False)['X'] 
norm_scvi_X_2 = sc.pp.normalize_total(AnnData(internal_scvi_X_2), target_sum=1e4, inplace=False)['X']
norm_avg_X = sc.pp.normalize_total(AnnData(internal_avg_X), target_sum=1e4, inplace=False)['X']
norm_imputed_X = sc.pp.normalize_total(AnnData(imputed_X), target_sum=1e4, inplace=False)['X']
norm_cascvi_X = sc.pp.normalize_total(AnnData(internal_cascvi_X), target_sum=1e4, inplace=False)['X']
norm_cascvi_X_2 = sc.pp.normalize_total(AnnData(internal_cascvi_X_2), target_sum=1e4, inplace=False)['X']

norm_internal_X.shape

(500, 1000)

In [189]:
from scipy import stats

data0 = internal_X[:, 0]
data1 = imputed_X[:, 0]

stats.pearsonr(data1, data0), stats.kendalltau(data1, data0), 

((0.713590996797604, 5.38753823209405e-79),
 KendalltauResult(correlation=0.47277619054185444, pvalue=6.927823772966586e-45))

## I. Sample-Sample Correlations

***1. Sample-Sample correlation (Without Normalization)***

We will use Scipy to compute a nonparametric rank correlation between the imputed and the groundtruth profiles. The correlation is based on the Spearman Correlation Coefficient.

In [190]:
data = {'groundtruth': internal_X.T, 'cascVI': imputed_X.T, 'scVI': internal_scvi_X_2.T,
        'Average': internal_avg_X.T , 'cascVI + Avg': internal_cascvi_X_2.T,
        'MP Oracle': internal_cascvi_X.T
        }
df1 = correlations(data, 'None', True)
df1.head(5)
plt.show()

***2. Sample-Sample correlation (With ScanPy Normalization)***

In [191]:
data = {'groundtruth': norm_internal_X.T, 'cascVI': norm_imputed_X.T, 'scVI': norm_scvi_X_2.T, 
        'Average': norm_avg_X.T , 'cascVI + Avg': norm_cascvi_X_2.T,
        'MP Oracle': norm_cascvi_X.T
        }

df2 = correlations(data, 'None', True)
df2.head(5)
plt.show()

/home/eecs/khalil.ouardini/Cassiopeia_Transcriptome/scvi/external/utils/metrics.py:90: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(1, 3, figsize=(20, 5), sharey=True)



## II. Gene-Gene Correlations

***2. Gene-Gene correlation (With Normalization)***

In [192]:
data = {'groundtruth': internal_X, 'cascVI': imputed_X, 'scVI': internal_scvi_X_2,
        'Average': internal_avg_X , 'cascVI + Avg': internal_cascvi_X_2,
        'MP Oracle': internal_cascvi_X
        }

df3 = correlations(data, 'None', True)
df3.head(5)
plt.show()

/home/eecs/khalil.ouardini/miniconda3/envs/scvi-env/lib/python3.7/site-packages/scipy/stats/stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/eecs/khalil.ouardini/miniconda3/envs/scvi-env/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


***2. Gene-Gene correlation (With Normalization)***

In [193]:
data = {'groundtruth': norm_internal_X, 'cascVI': norm_imputed_X, 'scVI': norm_scvi_X_2, 
        'Average': norm_avg_X , 'cascVI + Avg': norm_cascvi_X_2,
        'MP Oracle': norm_cascvi_X
        }

df4 = correlations(data, 'None', True)
df4.head(5)
plt.show()

/home/eecs/khalil.ouardini/miniconda3/envs/scvi-env/lib/python3.7/site-packages/scipy/stats/stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/eecs/khalil.ouardini/miniconda3/envs/scvi-env/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


***3. Gene-Gene correlation (With Rank Normalization)***

In [194]:
#data = {'groundtruth': norm_internal_X, 'cascVI': norm_imputed_X, 'scVI': norm_scvi_X_2, 
#        'Average': norm_avg_X , 'cascVI + Avg': norm_cascvi_X_2,
#        'MP Oracle': norm_cascvi_X
#        }

data = {'groundtruth': internal_X, 'cascVI': imputed_X, 'scVI': internal_scvi_X_2,
        'Average': internal_avg_X , 'cascVI + Avg': internal_cascvi_X_2,
        'MP Oracle': internal_cascvi_X
        }
        
df5 = correlations(data, 'rank', True)
df5.head(5)
plt.show()

/home/eecs/khalil.ouardini/miniconda3/envs/scvi-env/lib/python3.7/site-packages/scipy/stats/stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/eecs/khalil.ouardini/miniconda3/envs/scvi-env/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


### III. Table Summary

In [195]:
columns = ["Method", "Spearman CC", "Pearson CC", "Kendall Tau"]
data = [df1, df2, df3, df4, df5]
#data = [df2, df4]

data 
tables = [[] for i in range(len(data))]

#task = ["Sample-Sample (None)", "Sample-Sample (CPM)", "Gene-Gene (None)", 
           #"Gene-Gene(CPM)", "Gene-Gene (Rank)" ]

for (df, t) in zip(data, tables):
    for m in np.unique(df.Method):
        sub_df = np.round(df[df['Method'] == m].mean(), decimals=3)
        t.append([m, sub_df['Spearman CC'], sub_df['Pearson CC'], sub_df['Kendall Tau']])
        
# Create and style Data Frames
df_table1 = pd.DataFrame(tables[0], columns=columns)
df_table2 = pd.DataFrame(tables[1], columns=columns)
df_table3 = pd.DataFrame(tables[2], columns=columns)
df_table4 = pd.DataFrame(tables[3], columns=columns)
df_table5 = pd.DataFrame(tables[4], columns=columns)

In [196]:
print(" >>> Sample-Sample | No Normalization <<<")
df_table1.head(10)

 >>> Sample-Sample | No Normalization <<<


,Method,Spearman CC,Pearson CC,Kendall Tau
0,Average,0.904,0.944,0.824
1,MP Oracle,0.913,0.969,0.801
2,cascVI,0.911,0.966,0.795
3,cascVI + Avg,0.906,0.945,0.790
4,scVI,0.904,0.936,0.784


In [197]:
print(">>> Sample-Sample | CPM Normalization <<<")
df_table2.head(10)

>>> Sample-Sample | CPM Normalization <<<


,Method,Spearman CC,Pearson CC,Kendall Tau
0,Average,0.904,0.944,0.824
1,MP Oracle,0.913,0.969,0.801
2,cascVI,0.911,0.966,0.795
3,cascVI + Avg,0.906,0.945,0.790
4,scVI,0.904,0.936,0.784


In [198]:
print(">>> Gene-Gene | No Normalization <<<")
df_table3.head(10)

>>> Gene-Gene | No Normalization <<<


,Method,Spearman CC,Pearson CC,Kendall Tau
0,Average,0.620,0.680,0.529
1,MP Oracle,0.666,0.746,0.551
2,cascVI,0.586,0.569,0.460
3,cascVI + Avg,0.570,0.542,0.446
4,scVI,0.534,0.427,0.413


In [199]:
print(">>> Gene-Gene | CPM Normalization <<<")
df_table4.head(10)

>>> Gene-Gene | CPM Normalization <<<


,Method,Spearman CC,Pearson CC,Kendall Tau
0,Average,0.614,0.674,0.504
1,MP Oracle,0.654,0.735,0.529
2,cascVI,0.646,0.718,0.520
3,cascVI + Avg,0.630,0.692,0.503
4,scVI,0.604,0.590,0.479


In [200]:
print(">>> Gene-Gene | Rank Normalization <<<")
df_table5.head(10)

>>> Gene-Gene | Rank Normalization <<<


,Method,Spearman CC,Pearson CC,Kendall Tau
0,Average,0.620,0.620,0.529
1,MP Oracle,0.666,0.666,0.551
2,cascVI,0.586,0.586,0.460
3,cascVI + Avg,0.570,0.570,0.446
4,scVI,0.534,0.534,0.413


# 8. Latent Space Analysis

***cascVI***

In [201]:
# Merge internal nodes and leaves
#full_cascvi_latent = construct_latent(tree, cascvi_latent, imputed_z)


#print("CascVI latent space")
#plot_common_ancestor(tree,
#                     full_cascvi_latent,
#                     embedding='umap',
#                     give_labels=False
#                             )

***CascVI + avg***

In [202]:
#full_cascvi_latent_2 = construct_latent(tree, cascvi_latent, imputed_cascvi_2_z)

#print("CascVI + averaging latent space")
#plot_common_ancestor(tree,
#                     full_cascvi_latent_2,
#                     embedding='umap',
#                     give_labels=False
#                             )

***scVI***

In [203]:
# Merge internal nodes and leaves
#full_scvi_latent = construct_latent(tree, scvi_latent, imputed_scvi_2_z)

#print("scVI latent space")
#plot_common_ancestor(tree,
 #                full_scvi_latent,
 #                embedding='umap',
 #                give_labels=False
 #                   )

### k-NN purity

***LEAVES only***

In [204]:
#print("Leaves Only")
#data = {'groundtruth': leaves_z, 'scVI': scvi_latent,
#        'cascVI': cascvi_latent
#        }
#scores = knn_purity(max_neighbors=50,
#                    data=data,
#                    plot=True,
#                    save_fig='/home/eecs/khalil.ouardini/Cassiopeia_Transcriptome/scvi/tmp'
#                    )

*** Internal nodes only***

In [205]:
#print("Internal nodes Only")
#internal_z, internal_idx, internal_mu = get_internal(glm.z, glm.mu, tree)
#internal_scvi_z, _, _ = get_internal(full_scvi_latent, glm.mu, tree)
#internal_cascvi_z, _, _ = get_internal(full_cascvi_latent, glm.mu, tree)
#internal_cascvi_z_2, _, _ = get_internal(full_cascvi_latent_2, glm.mu, tree)

#data = {'groundtruth': internal_z, 'scVI + avg': internal_scvi_z,
#        'cascVI': internal_cascvi_z, 'cascVI + avg': internal_cascvi_z_2
#        }

#scores = knn_purity(max_neighbors=50,
#              data=data,
#              plot=True,
#              save_fig='/home/eecs/khalil.ouardini/Cassiopeia_Transcriptome/scvi/tmp/'
#              )

***Full tree***

In [206]:
#print("Full tree")
#data = {'groundtruth': glm.z, 'scVI + avg': full_scvi_latent,
#        'cascVI': full_cascvi_latent, 'cascVI + avg': full_cascvi_latent_2
#        }
#scores = knn_purity(max_neighbors=50,
#              data=data,
#              plot=True)

***Stratified k-NN purity***

In [207]:
#data = {'groundtruth': glm.z, 'scVI + avg': full_scvi_latent,
#        'cascVI': full_cascvi_latent, 'cascVI + avg': full_cascvi_latent_2
#        }

#for k in [2, 5, 10, 20, 35, 50]:
#    print("For {} neighbors".format(k))
#    if k == 10:
#        min_depth = 3
#    elif k == 20:
#        min_depth = 4
#    elif k == 35:
#        min_depth = 6
#    elif k == 50:
#        min_depth = 7
#    else:
#        min_depth = 2
#    scores = knn_purity_stratified(n_neighbors=k,
#                                   tree=tree,
#                                   data=data,
#                                   min_depth=min_depth,
#                                   plot=True)